单机版本

In [6]:
import ssl
ssl._create_default_https_context=ssl._create_unverified_context

In [7]:
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# nomalize
x_train, x_test = x_train / 255, x_test / 255
y_train_ohe = to_categorical(y_train, num_classes=10)
y_test_ohe = to_categorical(y_test, num_classes=10)


In [8]:
def cifar10_model(input_shape, num_classes):
    model = keras.Sequential(
        [
            tf.keras.Input(shape=input_shape),
            tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    # Compile model
    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return model


In [9]:
model = cifar10_model(input_shape=(32, 32, 3), num_classes=10)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]
)
model.fit(
    x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), batch_size=64, epochs=5
)


Epoch 1/5
782/782 [==============================] - 14s 18ms/step - loss: 1.6226 - categorical_accuracy: 0.4148 - val_loss: 1.3994 - val_categorical_accuracy: 0.5085
Epoch 2/5
782/782 [==============================] - 14s 17ms/step - loss: 1.3298 - categorical_accuracy: 0.5301 - val_loss: 1.1970 - val_categorical_accuracy: 0.5857
Epoch 3/5
782/782 [==============================] - 14s 18ms/step - loss: 1.2123 - categorical_accuracy: 0.5760 - val_loss: 1.1684 - val_categorical_accuracy: 0.5982
Epoch 4/5
782/782 [==============================] - 13s 17ms/step - loss: 1.1365 - categorical_accuracy: 0.6022 - val_loss: 1.0285 - val_categorical_accuracy: 0.6500
Epoch 5/5
782/782 [==============================] - 12s 15ms/step - loss: 1.0892 - categorical_accuracy: 0.6203 - val_loss: 1.0119 - val_categorical_accuracy: 0.6535


In [10]:
import secretflow as sf
sf.shutdown()
sf.init(['alice','bob'],address='local')
alice,bob = sf.PYU('alice'),sf.PYU('bob')


/usr/local/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-08-05 09:30:00,587	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=3.14gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-08-05 09:30:01,765	INFO worker.py:1724 -- Started a local Ray instance.


有两种数据导入的方式，一种是create_ndarray，还有一种是普通save/load（文件读入）

In [11]:
# create ndarray
from secretflow.utils.simulation.data.ndarray import create_ndarray

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# normalized x:
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
parts = {alice: 0.5, bob: 0.5}
fed_train_x = create_ndarray(x_train, parts=parts, axis=0)
fed_train_y = create_ndarray(y_train, parts=parts, axis=0)
fed_test_x = create_ndarray(x_test, parts=parts, axis=0)
fed_test_y = create_ndarray(y_test, parts=parts, axis=0)


In [12]:
fed_train_x.partition_shape()

{PYURuntime(alice): (25000, 32, 32, 3), PYURuntime(bob): (25000, 32, 32, 3)}

In [13]:
fed_train_y.partition_shape()

{PYURuntime(alice): (25000, 10), PYURuntime(bob): (25000, 10)}

In [14]:
# 文件读取
import numpy as np
from secretflow.data.ndarray import load

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
alice_train_x = x_train[:25000]
alice_train_y = y_train[:25000]
bob_train_x = x_train[25000:]
bob_train_y = y_train[25000:]
np.savez(
    "alice.npz",
    train_x=alice_train_x,
    train_y=alice_train_y,
    test_x=x_test,
    test_y=y_test,
)
np.savez(
    "bob.npz", train_x=bob_train_x, train_y=bob_train_y, test_x=x_test, test_y=y_test
)
fed_npz = load(
    {alice: "./alice.npz", bob: "./bob.npz"},
    partition_way=sf.data.ndarray.PartitionWay.HORIZONTAL,
)
fed_train_x, fed_train_y, fed_test_x, fed_test_y = (
    fed_npz["train_x"],
    fed_npz["train_y"],
    fed_npz["test_x"],
    fed_npz["test_y"],
)


In [15]:
fed_npz # 是个字典

{'train_x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00fa0>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa027a0>}, partition_way=<PartitionWay.HORIZONTAL: 'horizontal'>),
 'train_y': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa026b0>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00760>}, partition_way=<PartitionWay.HORIZONTAL: 'horizontal'>),
 'test_x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00640>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00400>}, partition_way=<PartitionWay.HORIZONTAL: 'horizontal'>),
 'test_y': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00040>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa01930>}, partition_

In [16]:
fed_train_y.partition_shape(), fed_train_x.partition_shape()

({PYURuntime(alice): (25000, 10), PYURuntime(bob): (25000, 10)},
 {PYURuntime(alice): (25000, 32, 32, 3), PYURuntime(bob): (25000, 32, 32, 3)})

In [17]:
# 定义模型
def create_model(input_shape,num_class):
    def be_wrapped_model():
        return cifar10_model(input_shape,num_class)
    return be_wrapped_model

In [18]:
from secretflow.ml.nn import FLModel
from secretflow.security.aggregation import SecureAggregator

fl_model = FLModel(
    device_list=[alice, bob],
    model=create_model(input_shape=(32, 32, 3), num_class=10),
    aggregator=SecureAggregator(bob, [alice, bob]),
    strategy="fed_avg_w",
    backend="tensorflow",
)


INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party bob.


In [19]:
fl_model.fit(
    fed_train_x,
    fed_train_y,
    validation_data=(fed_test_x, fed_test_y),
    epochs=5,
    aggregate_freq=10,
    batch_size=64,
)


INFO:root:FL Train Params: {'x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00fa0>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa027a0>}, partition_way=<PartitionWay.HORIZONTAL: 'horizontal'>), 'y': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa026b0>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00760>}, partition_way=<PartitionWay.HORIZONTAL: 'horizontal'>), 'batch_size': 64, 'batch_sampling_rate': None, 'epochs': 5, 'verbose': 1, 'callbacks': None, 'validation_data': (FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00640>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f127fa00400>}, partition_way=<PartitionWay.HORIZONTAL: 'horizontal'>), FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f127

Epoch 1/5


Train Processing: :   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 2/5


Train Processing: :   0%|          | 0/391 [00:00<?, ?it/s]

...

Epoch 4/5


Train Processing: :   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 5/5


Train Processing: : 100%|█████████▉| 390/391 [00:24<00:00, 15.70it/s, {'loss': 1.2742547, 'accuracy': 0.55394286, 'val_loss': 1.1391653, 'val_accuracy': 0.6128}]


{'global_history': {'loss': [2.2304354,
   1.6354717,
   1.4619344,
   1.3500277,
   1.2742547],
  'accuracy': [0.2492, 0.4116, 0.48272857, 0.5243, 0.55394286],
  'val_loss': [1.660353, 1.4303129, 1.2868077, 1.1805264, 1.1391653],
  'val_accuracy': [0.405, 0.5051, 0.5528, 0.5914, 0.6128]},
 'local_history': {'alice_loss': [2.2307498,
   1.6318861,
   1.4576604,
   1.3438071,
   1.26877],
  'alice_accuracy': [0.24756, 0.41145715, 0.4836857, 0.52625716, 0.55634284],
  'alice_val_loss': [1.660353, 1.4303129, 1.2868077, 1.1805264, 1.1391653],
  'alice_val_accuracy': [0.405, 0.5051, 0.5528, 0.5914, 0.6128],
  'bob_loss': [2.230121, 1.6390573, 1.4662086, 1.3562481, 1.2797395],
  'bob_accuracy': [0.25084, 0.41174287, 0.48177144, 0.52234286, 0.5515429],
  'bob_val_loss': [1.660353, 1.4303129, 1.2868077, 1.1805264, 1.1391653],
  'bob_val_accuracy': [0.405, 0.5051, 0.5528, 0.5914, 0.6128]}}

In [20]:
global_eval, local_eval = fl_model.evaluate(fed_test_x, fed_test_y)
for e in global_eval:
    print(e.name, e.result())


val_loss tf.Tensor(1.1391659, shape=(), dtype=float32)
val_accuracy tf.Tensor(0.6128, shape=(), dtype=float32)


In [21]:
pred_y = fl_model.predict(fed_test_x, batch_size=32)
model_path = {alice: "./alice_model", bob: "./bob_model"}
fl_model.save_model(model_path)


In [22]:
new_fl_model=FLModel(
device_list=[alice,bob],
model=create_model(input_shape=(32,32,3),num_class=10),
aggregator=SecureAggregator(bob,[alice,bob]),
strategy="fed_avg_g",
backend="tensorflow"
)
new_fl_model.load_model(model_path)

INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_g.PYUFedAvgG'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_g.PYUFedAvgG'> with party bob.
INFO:root:load model success


In [23]:
new_fl_model.evaluate(fed_test_x,fed_test_y)

([Mean(name='loss', total=22783.318, count=20000.0),
  Mean(name='accuracy', total=12256.0, count=20000.0)],
 {'alice': [Mean(name='loss', total=11391.659, count=10000.0),
   Mean(name='accuracy', total=6128.0, count=10000.0)],
  'bob': [Mean(name='loss', total=11391.659, count=10000.0),
   Mean(name='accuracy', total=6128.0, count=10000.0)]})

In [25]:
type(fed_test_x), type(fed_test_y)

(secretflow.data.ndarray.ndarray.FedNdarray,
 secretflow.data.ndarray.ndarray.FedNdarray)

In [4]:
# new_fl_model.fit(fed_train_x,fed_train_y)